In [1]:
from spectrum import SfgAverager
from orm import WorkDatabaseWizard
from sqlalchemy.orm.exc import NoResultFound

wz = WorkDatabaseWizard()

sml = wz.session.query(wz.samples).filter(wz.samples.type.in_(("s", "p")))
deep = wz.session.query(wz.samples).filter(wz.samples.type == "deep")

sml_june = sml.filter(wz.samples.sample_hash.like('06%'))
sml_september = sml.filter(wz.samples.sample_hash.like('09%'))

deep_june = deep.filter(wz.samples.sample_hash.like('06%'))
deep_september = deep.filter(wz.samples.sample_hash.like('09%'))

def get_gasex_average_spectrum(wz, query):
    
    samples = query.all()
    specs = wz.session.query(wz.gasex_sfg)
    to_average = []
    for sample in samples:
        try:
            spec = wz.session.query(wz.gasex_sfg).filter(wz.gasex_sfg.sample_hash == sample.sample_hash).one()
            spec_raw = wz.session.query(wz.sfg).filter(wz.sfg.name == spec.name).one()
            spec = wz.construct_sfg(spec_raw)
            to_average.append(spec)
        except NoResultFound:
            pass
    # requiere DPPC references!
    
    
    
get_average_spectrum(wz, sml_june)      

In [ ]:
wz.session.query

In [3]:
# test = wz.session.query(wz.sfg).filter(wz.sfg.name == "20121204_SFG-DPPC-8_31muL-(1_04mM)-#1").one()
# test = wz.construct_sfg(test)
# test.y = test.raw_intensity
from spectrum import SfgAverager

from sqlalchemy.exc import IntegrityError
def get_gasex_references(wz):
    dates = {}
    q = wz.session.query(wz.sfg).filter(wz.sfg.measured_time.between("2018-06-01", "2018-12-31"))
    q = q.filter(wz.sfg.name.like("%DPPC%"))

    for item in q.all():
        date = item.measured_time.date()
        if date not in dates:
            dates[date] = [wz.construct_sfg(item)]
        else:
            dates[date].append(wz.construct_sfg(item))
        
    for key in dates:
        dates[key] = SfgAverager(dates[key]).integral
    
    return dates
            
def persist_gasex_references(wz):
    
    d = get_gasex_references(wz)
    for key in d:
        try:
            measurement_day = wz.measurement_days()
            measurement_day.date = key
            measurement_day.dppc_integral = d[key]
            print(d[key])
            wz.session.add(measurement_day)
            wz.session.commit()
        except IntegrityError:
            print("FAIL")
            wz.session.rollback()


persist_gasex_references(wz)



0.3761730704028462
0.21056810431674822
0.5126991466660782
0.38187333270529833
0.33891177108448783
0.32124991117600005
0.3935388488704654
0.28544983809875496
0.36793703944665757
0.25439224531724686
0.3361405098616312
0.4580174563336482
0.3707526510908267
0.2815079167213334
0.5309081361480653
0.6883923488386838
0.7811687405726224
0.6833097735492151
0.6414385366547648
0.692118326966174
0.6658327413832785
0.6833213932804724
0.6353282992184789
0.7389034374091885
0.7328498013438296
0.7448235227107485
0.7874878346275154
0.5020913705013393
FAIL
0.7034204289510111
FAIL
0.26277499597339843
FAIL
0.37736950409705544
FAIL
0.33072819181124563
FAIL
